In [36]:
#Imports
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import numpy as np

In [37]:
# Watermark
print('Nathan Young\nJunior Data Analyst\nCenter for the Study of Free Enterprise')
%load_ext watermark
%watermark -a "Western Carolina University" -u -d -p pandas

Nathan Young
Junior Data Analyst
Center for the Study of Free Enterprise
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Western Carolina University 
last updated: 2020-03-14 

pandas 1.0.1


In [38]:
# Create backups
#df_backup = pd.read_csv('./Updates/STG_ZLLW_County_MedianValuePerSqft_AllHomes.txt')
#df_backup.to_csv('./Backups/STG_ZLLW_County_MedianValuePerSqft_AllHomes_BACKUP.txt')

In [39]:
#Load Land data
df_mvsf = pd.read_csv('http://files.zillowstatic.com/research/public/County/County_MedianValuePerSqft_AllHomes.csv', 
                      encoding='ISO-8859-1')

#Display table to ensure data loaded correctly
df_mvsf.head()

,RegionID,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,1996-04,1996-05,1996-06,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
0,3101,Los Angeles County,CA,Los Angeles-Long Beach-Anaheim,6,37,1,1034.0,1034.0,1034.0,...,1035,1035,1035,1035,1035,1035,1035,1035,1035,1035
1,139,Cook County,IL,Chicago-Naperville-Elgin,17,31,2,460.0,460.0,460.0,...,460,460,460,460,460,460,460,460,460,460
2,1090,Harris County,TX,Houston-The Woodlands-Sugar Land,48,201,3,165.0,165.0,165.0,...,165,165,165,165,165,165,165,165,165,165
3,2402,Maricopa County,AZ,Phoenix-Mesa-Scottsdale,4,13,4,455.0,455.0,455.0,...,455,455,455,455,455,455,455,455,455,455
4,2841,San Diego County,CA,San Diego-Carlsbad,6,73,5,908.0,908.0,908.0,...,908,908,908,908,908,908,908,908,908,908


In [40]:
#Filter data to NC
filter1 = df_mvsf['State'] == "NC"
df_mvsf_nc = df_mvsf[filter1]

#Check to ensure filter worked
df_mvsf_nc.head(5)

,RegionID,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,1996-04,1996-05,1996-06,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
48,3102,Mecklenburg County,NC,Charlotte-Concord-Gastonia,37,119,49,188.0,188.0,188.0,...,188,188,188,188,188,188,188,188,188,188
55,324,Wake County,NC,Raleigh,37,183,56,220.0,220.0,220.0,...,220,220,220,220,220,220,220,220,220,220
134,2312,Guilford County,NC,Greensboro-High Point,37,81,135,144.0,144.0,144.0,...,144,144,144,144,144,144,144,144,144,144
183,1712,Forsyth County,NC,Winston-Salem,37,67,184,195.0,195.0,195.0,...,195,195,195,195,195,195,195,195,195,195
198,2912,Cumberland County,NC,Fayetteville,37,51,199,146.0,146.0,146.0,...,146,146,146,146,146,146,146,146,146,146


In [41]:
#View data types of dataframe
df_mvsf_nc.dtypes

RegionID          int64
RegionName       object
State            object
Metro            object
StateCodeFIPS     int64
                  ...  
2019-09           int64
2019-10           int64
2019-11           int64
2019-12           int64
2020-01           int64
Length: 293, dtype: object

In [42]:
#Change MunicipalCodeFIPS dtype to add leading 0's
df_mvsf_nc.loc[ :, 'MunicipalCodeFIPS'] = df_mvsf_nc['MunicipalCodeFIPS'].astype(str)
df_mvsf_nc.dtypes

C:\Users\natha\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


RegionID          int64
RegionName       object
State            object
Metro            object
StateCodeFIPS     int64
                  ...  
2019-09           int64
2019-10           int64
2019-11           int64
2019-12           int64
2020-01           int64
Length: 293, dtype: object

In [43]:
#Add leading 0's and check to ensure they were added
df_mvsf_nc.loc[ :, 'MunicipalCodeFIPS'] = df_mvsf_nc['MunicipalCodeFIPS'].str.zfill(3)
df_mvsf_nc.head(5)

,RegionID,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,1996-04,1996-05,1996-06,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
48,3102,Mecklenburg County,NC,Charlotte-Concord-Gastonia,37,119,49,188.0,188.0,188.0,...,188,188,188,188,188,188,188,188,188,188
55,324,Wake County,NC,Raleigh,37,183,56,220.0,220.0,220.0,...,220,220,220,220,220,220,220,220,220,220
134,2312,Guilford County,NC,Greensboro-High Point,37,081,135,144.0,144.0,144.0,...,144,144,144,144,144,144,144,144,144,144
183,1712,Forsyth County,NC,Winston-Salem,37,067,184,195.0,195.0,195.0,...,195,195,195,195,195,195,195,195,195,195
198,2912,Cumberland County,NC,Fayetteville,37,051,199,146.0,146.0,146.0,...,146,146,146,146,146,146,146,146,146,146


In [44]:
# Set Index to Region Name
df_mvsf_nc.set_index(df_mvsf_nc['RegionName'], inplace = True)
df_mvsf_nc

,RegionID,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,1996-04,1996-05,1996-06,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
RegionName,,,,,,,,,,,,,,,,,,,,,
Mecklenburg County,3102,Mecklenburg County,NC,Charlotte-Concord-Gastonia,37,119,49,188.0,188.0,188.0,...,188,188,188,188,188,188,188,188,188,188
Wake County,324,Wake County,NC,Raleigh,37,183,56,220.0,220.0,220.0,...,220,220,220,220,220,220,220,220,220,220
Guilford County,2312,Guilford County,NC,Greensboro-High Point,37,081,135,144.0,144.0,144.0,...,144,144,144,144,144,144,144,144,144,144
Forsyth County,1712,Forsyth County,NC,Winston-Salem,37,067,184,195.0,195.0,195.0,...,195,195,195,195,195,195,195,195,195,195
Cumberland County,2912,Cumberland County,NC,Fayetteville,37,051,199,146.0,146.0,146.0,...,146,146,146,146,146,146,146,146,146,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jones County,575,Jones County,NC,New Bern,37,103,2402,NaN,NaN,NaN,...,68,68,68,68,68,68,68,68,68,68
Camden County,910,Camden County,NC,Elizabeth City,37,029,2418,NaN,NaN,NaN,...,556,556,556,556,556,556,556,556,556,556
Graham County,1049,Graham County,NC,NaN,37,075,2502,NaN,NaN,NaN,...,79,79,79,79,79,79,79,79,79,79


In [45]:
# Drop Region Name column
df_mvsf_nc.drop('RegionName', axis = 1, inplace = True)
df_mvsf_nc

C:\Users\natha\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,RegionID,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
RegionName,,,,,,,,,,,,,,,,,,,,,
Mecklenburg County,3102,NC,Charlotte-Concord-Gastonia,37,119,49,188.0,188.0,188.0,188.0,...,188,188,188,188,188,188,188,188,188,188
Wake County,324,NC,Raleigh,37,183,56,220.0,220.0,220.0,220.0,...,220,220,220,220,220,220,220,220,220,220
Guilford County,2312,NC,Greensboro-High Point,37,081,135,144.0,144.0,144.0,144.0,...,144,144,144,144,144,144,144,144,144,144
Forsyth County,1712,NC,Winston-Salem,37,067,184,195.0,195.0,195.0,195.0,...,195,195,195,195,195,195,195,195,195,195
Cumberland County,2912,NC,Fayetteville,37,051,199,146.0,146.0,146.0,146.0,...,146,146,146,146,146,146,146,146,146,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jones County,575,NC,New Bern,37,103,2402,NaN,NaN,NaN,NaN,...,68,68,68,68,68,68,68,68,68,68
Camden County,910,NC,Elizabeth City,37,029,2418,NaN,NaN,NaN,NaN,...,556,556,556,556,556,556,556,556,556,556
Graham County,1049,NC,NaN,37,075,2502,NaN,NaN,NaN,NaN,...,79,79,79,79,79,79,79,79,79,79


In [46]:
#Save to csv file for export in Excel
#df_mvsf_nc.to_csv('./Updates/STG_ZLLW_County_MedianValuePerSqft_AllHomes.txt', sep = '\t')

In [47]:
#Reset Index for upload to database
df_mvsf_nc = df_mvsf_nc.reset_index()    

In [48]:
#Fill NaN values for upload to database
df_mvsf_nc['Metro'] = df_mvsf_nc['Metro'].replace(np.nan,'', regex=True)

column_list = df_mvsf_nc.columns.values
for i in column_list:
    df_mvsf_nc.loc[df_mvsf_nc[i].isnull(),i]=0

In [49]:
#Connect to database and create cursor
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=TITANIUM-BOOK;'
                      'Database=DataDashboard;'
                      'Trusted_Connection=yes;',
                    autocommit=True)

c = con.cursor()

In [50]:
#Drop old backup table
#c.execute('drop table STG_ZLLW_County_MedianValuePerSqft_AllHomes_BACKUP')

In [51]:
#Create new backup
c.execute('''sp_rename 'dbo.STG_ZLLW_County_MedianValuePerSqft_AllHomes','STG_ZLLW_County_MedianValuePerSqft_AllHomes_BACKUP';''')

In [52]:
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON

SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_ZLLW_County_MedianValuePerSqft_AllHomes](
	[RegionName] [varchar](19) NULL,
	[RegionID] [smallint] NULL,
	[State] [varchar](2) NULL,
	[Metro] [varchar](38) NULL,
	[StateCodeFIPS] [varchar](2) NULL,
	[MunicipalCodeFIPS] [varchar](3) NULL,
	[SizeRank] [smallint] NULL,
	[1996-04] [float] NULL,
	[1996-05] [float] NULL,
	[1996-06] [float] NULL,
	[1996-07] [float] NULL,
	[1996-08] [float] NULL,
	[1996-09] [float] NULL,
	[1996-10] [float] NULL,
	[1996-11] [float] NULL,
	[1996-12] [float] NULL,
	[1997-01] [float] NULL,
	[1997-02] [float] NULL,
	[1997-03] [float] NULL,
	[1997-04] [float] NULL,
	[1997-05] [float] NULL,
	[1997-06] [float] NULL,
	[1997-07] [float] NULL,
	[1997-08] [float] NULL,
	[1997-09] [float] NULL,
	[1997-10] [float] NULL,
	[1997-11] [float] NULL,
	[1997-12] [float] NULL,
	[1998-01] [float] NULL,
	[1998-02] [float] NULL,
	[1998-03] [float] NULL,
	[1998-04] [float] NULL,
	[1998-05] [float] NULL,
	[1998-06] [float] NULL,
	[1998-07] [float] NULL,
	[1998-08] [float] NULL,
	[1998-09] [float] NULL,
	[1998-10] [float] NULL,
	[1998-11] [float] NULL,
	[1998-12] [float] NULL,
	[1999-01] [float] NULL,
	[1999-02] [float] NULL,
	[1999-03] [float] NULL,
	[1999-04] [float] NULL,
	[1999-05] [float] NULL,
	[1999-06] [float] NULL,
	[1999-07] [float] NULL,
	[1999-08] [float] NULL,
	[1999-09] [float] NULL,
	[1999-10] [float] NULL,
	[1999-11] [float] NULL,
	[1999-12] [float] NULL,
	[2000-01] [float] NULL,
	[2000-02] [float] NULL,
	[2000-03] [float] NULL,
	[2000-04] [float] NULL,
	[2000-05] [float] NULL,
	[2000-06] [float] NULL,
	[2000-07] [float] NULL,
	[2000-08] [float] NULL,
	[2000-09] [float] NULL,
	[2000-10] [float] NULL,
	[2000-11] [float] NULL,
	[2000-12] [float] NULL,
	[2001-01] [float] NULL,
	[2001-02] [float] NULL,
	[2001-03] [float] NULL,
	[2001-04] [float] NULL,
	[2001-05] [float] NULL,
	[2001-06] [float] NULL,
	[2001-07] [float] NULL,
	[2001-08] [float] NULL,
	[2001-09] [float] NULL,
	[2001-10] [float] NULL,
	[2001-11] [float] NULL,
	[2001-12] [float] NULL,
	[2002-01] [float] NULL,
	[2002-02] [float] NULL,
	[2002-03] [float] NULL,
	[2002-04] [float] NULL,
	[2002-05] [float] NULL,
	[2002-06] [float] NULL,
	[2002-07] [float] NULL,
	[2002-08] [float] NULL,
	[2002-09] [float] NULL,
	[2002-10] [float] NULL,
	[2002-11] [float] NULL,
	[2002-12] [float] NULL,
	[2003-01] [float] NULL,
	[2003-02] [float] NULL,
	[2003-03] [float] NULL,
	[2003-04] [float] NULL,
	[2003-05] [float] NULL,
	[2003-06] [float] NULL,
	[2003-07] [float] NULL,
	[2003-08] [float] NULL,
	[2003-09] [float] NULL,
	[2003-10] [float] NULL,
	[2003-11] [float] NULL,
	[2003-12] [float] NULL,
	[2004-01] [float] NULL,
	[2004-02] [float] NULL,
	[2004-03] [float] NULL,
	[2004-04] [float] NULL,
	[2004-05] [float] NULL,
	[2004-06] [float] NULL,
	[2004-07] [float] NULL,
	[2004-08] [float] NULL,
	[2004-09] [float] NULL,
	[2004-10] [float] NULL,
	[2004-11] [float] NULL,
	[2004-12] [float] NULL,
	[2005-01] [float] NULL,
	[2005-02] [float] NULL,
	[2005-03] [float] NULL,
	[2005-04] [float] NULL,
	[2005-05] [float] NULL,
	[2005-06] [float] NULL,
	[2005-07] [float] NULL,
	[2005-08] [float] NULL,
	[2005-09] [float] NULL,
	[2005-10] [float] NULL,
	[2005-11] [float] NULL,
	[2005-12] [float] NULL,
	[2006-01] [float] NULL,
	[2006-02] [float] NULL,
	[2006-03] [float] NULL,
	[2006-04] [float] NULL,
	[2006-05] [float] NULL,
	[2006-06] [float] NULL,
	[2006-07] [float] NULL,
	[2006-08] [float] NULL,
	[2006-09] [float] NULL,
	[2006-10] [float] NULL,
	[2006-11] [float] NULL,
	[2006-12] [float] NULL,
	[2007-01] [float] NULL,
	[2007-02] [float] NULL,
	[2007-03] [float] NULL,
	[2007-04] [float] NULL,
	[2007-05] [float] NULL,
	[2007-06] [float] NULL,
	[2007-07] [float] NULL,
	[2007-08] [float] NULL,
	[2007-09] [float] NULL,
	[2007-10] [float] NULL,
	[2007-11] [float] NULL,
	[2007-12] [float] NULL,
	[2008-01] [float] NULL,
	[2008-02] [float] NULL,
	[2008-03] [float] NULL,
	[2008-04] [float] NULL,
	[2008-05] [float] NULL,
	[2008-06] [float] NULL,
	[2008-07] [float] NULL,
	[2008-08] [float] NULL,
	[2008-09] [float] NULL,
	[2008-10] [float] NULL,
	[2008-11] [float] NULL,
	[2008-12] [float] NULL,
	[2009-01] [float] NULL,
	[2009-02] [float] NULL,
	[2009-03] [float] NULL,
	[2009-04] [float] NULL,
	[2009-05] [float] NULL,
	[2009-06] [float] NULL,
	[2009-07] [float] NULL,
	[2009-08] [float] NULL,
	[2009-09] [float] NULL,
	[2009-10] [float] NULL,
	[2009-11] [float] NULL,
	[2009-12] [float] NULL,
	[2010-01] [float] NULL,
	[2010-02] [float] NULL,
	[2010-03] [float] NULL,
	[2010-04] [float] NULL,
	[2010-05] [float] NULL,
	[2010-06] [float] NULL,
	[2010-07] [float] NULL,
	[2010-08] [float] NULL,
	[2010-09] [float] NULL,
	[2010-10] [float] NULL,
	[2010-11] [float] NULL,
	[2010-12] [float] NULL,
	[2011-01] [float] NULL,
	[2011-02] [float] NULL,
	[2011-03] [float] NULL,
	[2011-04] [float] NULL,
	[2011-05] [float] NULL,
	[2011-06] [float] NULL,
	[2011-07] [float] NULL,
	[2011-08] [float] NULL,
	[2011-09] [float] NULL,
	[2011-10] [float] NULL,
	[2011-11] [float] NULL,
	[2011-12] [float] NULL,
	[2012-01] [float] NULL,
	[2012-02] [float] NULL,
	[2012-03] [float] NULL,
	[2012-04] [float] NULL,
	[2012-05] [float] NULL,
	[2012-06] [float] NULL,
	[2012-07] [float] NULL,
	[2012-08] [float] NULL,
	[2012-09] [float] NULL,
	[2012-10] [float] NULL,
	[2012-11] [float] NULL,
	[2012-12] [float] NULL,
	[2013-01] [float] NULL,
	[2013-02] [float] NULL,
	[2013-03] [float] NULL,
	[2013-04] [float] NULL,
	[2013-05] [float] NULL,
	[2013-06] [float] NULL,
	[2013-07] [float] NULL,
	[2013-08] [float] NULL,
	[2013-09] [float] NULL,
	[2013-10] [float] NULL,
	[2013-11] [float] NULL,
	[2013-12] [float] NULL,
	[2014-01] [float] NULL,
	[2014-02] [float] NULL,
	[2014-03] [float] NULL,
	[2014-04] [float] NULL,
	[2014-05] [float] NULL,
	[2014-06] [float] NULL,
	[2014-07] [float] NULL,
	[2014-08] [float] NULL,
	[2014-09] [float] NULL,
	[2014-10] [float] NULL,
	[2014-11] [float] NULL,
	[2014-12] [float] NULL,
	[2015-01] [float] NULL,
	[2015-02] [float] NULL,
	[2015-03] [float] NULL,
	[2015-04] [float] NULL,
	[2015-05] [float] NULL,
	[2015-06] [float] NULL,
	[2015-07] [float] NULL,
	[2015-08] [float] NULL,
	[2015-09] [float] NULL,
	[2015-10] [float] NULL,
	[2015-11] [float] NULL,
	[2015-12] [float] NULL,
	[2016-01] [float] NULL,
	[2016-02] [float] NULL,
	[2016-03] [float] NULL,
	[2016-04] [float] NULL,
	[2016-05] [float] NULL,
	[2016-06] [float] NULL,
	[2016-07] [float] NULL,
	[2016-08] [float] NULL,
	[2016-09] [float] NULL,
	[2016-10] [float] NULL,
	[2016-11] [float] NULL,
	[2016-12] [float] NULL,
	[2017-01] [float] NULL,
	[2017-02] [float] NULL,
	[2017-03] [float] NULL,
	[2017-04] [float] NULL,
	[2017-05] [float] NULL,
	[2017-06] [float] NULL,
	[2017-07] [float] NULL,
	[2017-08] [float] NULL,
	[2017-09] [float] NULL,
	[2017-10] [float] NULL,
	[2017-11] [float] NULL,
	[2017-12] [float] NULL,
	[2018-01] [float] NULL,
	[2018-02] [float] NULL,
	[2018-03] [float] NULL,
	[2018-04] [float] NULL,
	[2018-05] [float] NULL,
	[2018-06] [float] NULL,
	[2018-07] [float] NULL,
	[2018-08] [float] NULL,
	[2018-09] [float] NULL,
	[2018-10] [float] NULL,
	[2018-11] [float] NULL,
	[2018-12] [float] NULL,
	[2019-01] [float] NULL,
	[2019-02] [float] NULL,
	[2019-03] [float] NULL,
	[2019-04] [float] NULL,
	[2019-05] [float] NULL,
	[2019-06] [float] NULL,
	[2019-07] [float] NULL,
	[2019-08] [float] NULL,
	[2019-09] [float] NULL,
	[2019-10] [float] NULL,
	[2019-11] [float] NULL,
	[2019-12] [float] NULL,
    [2020-01] [float] NULL,
    [2020-02] [float] NULL,
    [2020-03] [float] NULL,
    [2020-04] [float] NULL,
    [2020-05] [float] NULL,
    [2020-06] [float] NULL,
    [2020-07] [float] NULL,
    [2020-08] [float] NULL,
    [2020-09] [float] NULL,
    [2020-10] [float] NULL,
    [2020-11] [float] NULL,
    [2020-12] [float] NULL,
    [2021-01] [float] NULL,
    [2021-02] [float] NULL,
    [2021-03] [float] NULL,
    [2021-04] [float] NULL,
    [2021-05] [float] NULL,
    [2021-06] [float] NULL,
    [2021-07] [float] NULL,
    [2021-08] [float] NULL,
    [2021-09] [float] NULL,
    [2021-10] [float] NULL,
    [2021-11] [float] NULL,
    [2021-12] [float] NULL,
    [2022-01] [float] NULL,
    [2022-02] [float] NULL,
    [2022-03] [float] NULL,
    [2022-04] [float] NULL,
    [2022-05] [float] NULL,
    [2022-06] [float] NULL,
    [2022-07] [float] NULL,
    [2022-08] [float] NULL,
    [2022-09] [float] NULL,
    [2022-10] [float] NULL,
    [2022-11] [float] NULL,
    [2022-12] [float] NULL
) ON [PRIMARY]''')

In [53]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#warning: discard old table if exists
df_mvsf_nc.to_sql('STG_ZLLW_County_MedianValuePerSqft_AllHomes', con=engine, if_exists='replace', index=False)